In [190]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [191]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [192]:
T = 10*24;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [193]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1)), PeriodLength = 240)

In [194]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;

ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MWh of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MWh of H2
electrolyzer_vom_cost = 0.0;


In [195]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [196]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [197]:
solar_pv = Transformation{Electricity}(;
    id = :SolarPV,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    #### Note that this transformation does not have a stoichiometry balance because the sunshine is exogenous :-)
)
solar_pv.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = solar_pv,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
capacity_factor = solar_capacity_factor,
investment_cost = solar_inv_cost,
fixed_om_cost = solar_fom_cost,
constraints = [Macro.CapacityConstraint()]
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electricity}(:SolarPV, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:E => TEdge{Electricity}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.5176, 0.3503, 0.1105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:

We can also assigne a battery storage to the node:

In [198]:
battery = Transformation{Storage}(;
id = :battery,
stoichiometry_balance_names = [:storage],
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
can_expand = true,
can_retire = false,
existing_capacity_storage = 0.0,
investment_cost_storage = battery_inv_cost_storage,
fixed_om_cost_storage = battery_fom_cost_storage,
min_duration = 1,
max_duration = 10,
constraints = [Macro.StorageCapacityConstraint(),Macro.StoichiometryBalanceConstraint()],
discharge_capacity_edge = :discharge
)

battery.TEdges[:discharge] = TEdge{Electricity}(;
id = :discharge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
investment_cost = battery_inv_cost,
fixed_om_cost = battery_fom_cost,
variable_om_cost = battery_vom_cost,
st_coeff = Dict(:storage=>1/battery_eff_down),
constraints = [Macro.CapacityConstraint()]
)

battery.TEdges[:charge] = TEdge{Electricity}(;
id = :charge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :input,
has_planning_variables = false,
st_coeff = Dict(:storage=>battery_eff_up)
)


TEdge{Electricity}(:charge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Storage}(:battery, 1:1:240, StepRange{Int64, Int64}[1:1:240], [:storage], Dict{Symbol, TEdge}(:charge => TEdge{Electricity}(#= circular reference @-3 =#), :discharge => TEdge{Electricity}(:discharge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [500

We also consider a hydrogen demand node:

In [199]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [200]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

In [201]:
NGImport = Transformation{NaturalGas}(;
id = :NGImport,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
#### Note that this transformation does not have a stoichiometry balance because we are modeling exogenous inflow of NG
)

NGImport.TEdges[:ng_source] = TEdge{NaturalGas}(;
id = :ng_source,
node = ng_node,
transformation = NGImport,
direction = :output,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
has_planning_variables = false,
price = ng_fuel_price,
)

TEdge{NaturalGas}(:ng_source, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGas}(:NGImport, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:ng_source => TEdge{NaturalGas}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.016107833

In [202]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
)

Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

In [203]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.)

This transformation has 3 edges: each edge carries a specific commodity flow

In [204]:
ngcc.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_heatrate,:emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost =ngcc_vom_cost,
constraints = [Macro.CapacityConstraint()]
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:E => TEdge{Electricity}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 250.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240

In [205]:
ngcc.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_fuel_CO2)
)

TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(#= circular reference @-3 =#), :E => TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Floa

In [206]:
ngcc.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
    )

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[]), Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(#= circular reference @-3 =#), :input, false, false, false, 1.0, F

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [207]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.)

In [208]:
electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(:H2 => TEdge{Hydrogen}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 66

In [209]:
electrolyzer.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(:H2 => TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{An

Now that we have implemented different technologies, we can create the JuMP model.

In [210]:
nodes = [e_node;h2_node;ng_node;co2_node];
system = [nodes;solar_pv; battery; ngcc; electrolyzer; NGImport];

In [211]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [212]:
Macro.add_planning_variables!.(system,model);

In [213]:
Macro.add_operation_variables!.(system, model);

In [214]:
Macro.add_all_model_constraints!.(system, model);

┌ Warning: MaxNonServedDemandConstraint required for a node that does not have a non-served demand variable so MACRO will not create this constraint
└ @ Macro c:\Users\pecci\Code\MACRO-dev\src\node.jl:92
┌ Warning: MaxNonServedDemandConstraint required for a node that does not have a non-served demand variable so MACRO will not create this constraint
└ @ Macro c:\Users\pecci\Code\MACRO-dev\src\node.jl:92
┌ Warning: MaxNonServedDemandConstraint required for a node that does not have a non-served demand variable so MACRO will not create this constraint
└ @ Macro c:\Users\pecci\Code\MACRO-dev\src\node.jl:92


In [215]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [216]:
println(ngcc.constraints[1].constraint_ref[:energy,5])

-vFLOW_NGCC_NG[5] + 2.1775180500999998 vFLOW_NGCC_E[5] == 0


In [217]:
println(ngcc.constraints[1].constraint_ref[:emissions,5])

-0.181048235160161 vFLOW_NGCC_NG[5] + vFLOW_NGCC_CO2[5] == 0


In [218]:
println(electrolyzer.constraints[1].constraint_ref[:energy,5])

vFLOW_Electrolyzer_H2[5] - 0.7406666666666667 vFLOW_Electrolyzer_E[5] == 0


In [219]:
using Gurobi

In [220]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-18


In [221]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2888 rows, 2416 columns and 6590 nonzeros
Model fingerprint: 0xb41a8392
Coefficient statistics:
  Matrix range     [2e-02, 3e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+04]
Presolve removed 1596 rows and 1362 columns
Presolve time: 0.00s
Presolved: 1292 rows, 1054 columns, 3394 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4370951e+07   3.815610e+06   0.000000e+00      0s
     951    1.2233098e+09   0.000000e+00   0.000000e+00      0s

Solved in 951 iterations and 0.01 seconds (0.01 work units)
Optimal objective  1.223309829e+09

User-callback calls 997, time in user-callback 0.00 sec


The installed solar capacity in MW is:

In [222]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

0.0

The installed battery capacity in MW is:

In [223]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

297.4500000000007

The installed NGCC capacity in MW is:

In [224]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

13696.3

The resulting CO2 emissions in tons are:

In [225]:
base_emissions  = Macro.value(sum(Macro.net_balance(co2_node)[t] for t in time_interval(CO2)))

976229.8341102058

However, if we add a CO2 cap:

In [226]:
merge!(co2_node.rhs_policy,Dict(Macro.CO2CapConstraint=>0.5*base_emissions))
append!(co2_node.constraints,[Macro.CO2CapConstraint()])
Macro.add_model_constraint!(co2_node.constraints[1],co2_node,model)

1-dimensional DenseAxisArray{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, JuMP.ScalarShape},1,...} with index sets:
    Dimension 1, [1]
And data, a 1-element Vector{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, JuMP.ScalarShape}}:
 vFLOW_NGCC_CO2[1] + vFLOW_NGCC_CO2[2] + vFLOW_NGCC_CO2[3] + vFLOW_NGCC_CO2[4] + vFLOW_NGCC_CO2[5] + vFLOW_NGCC_CO2[6] + vFLOW_NGCC_CO2[7] + vFLOW_NGCC_CO2[8] + vFLOW_NGCC_CO2[9] + vFLOW_NGCC_CO2[10] + vFLOW_NGCC_CO2[11] + vFLOW_NGCC_CO2[12] + vFLOW_NGCC_CO2[13] + vFLOW_NGCC_CO2[14] + vFLOW_NGCC_CO2[15] + vFLOW_NGCC_CO2[16] + vFLOW_NGCC_CO2[17] + vFLOW_NGCC_CO2[18] + vFLOW_NGCC_CO2[19] + vFLOW_NGCC_CO2[20] + vFLOW_NGCC_CO2[21] + vFLOW_NGCC_CO2[22] + vFLOW_NGCC_CO2[23] + vFLOW_NGCC_CO2[24] + vFLOW_NGCC_CO2[25] + vFLOW_NGCC_CO2[26] + vFLOW_NGCC_CO2[27] + vF

In [227]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2889 rows, 2416 columns and 6830 nonzeros
Coefficient statistics:
  Matrix range     [2e-02, 3e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 5e+05]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2233098e+09   4.881149e+05   0.000000e+00      0s
     300    5.8631392e+09   0.000000e+00   0.000000e+00      0s

Solved in 300 iterations and 0.01 seconds (0.04 work units)
Optimal objective  5.863139235e+09

User-callback calls 320, time in user-callback 0.00 sec


Now the emissions are:

In [233]:
Macro.value(sum(Macro.net_balance(co2_node)[t] for t in time_interval(CO2)))

488114.91705510195

and the solar capacity is:

In [228]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

35396.286894633326

The installed battery capacity is:

In [229]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

5743.072008322795

The installed NG capacity is:

In [230]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

8250.677991677205

Let's validate the battery modeling:

In [231]:
diff_storage_level = zeros(T);
for t in 1:T
    if t==1
        diff_storage_level[t] = Macro.value(Macro.storage_level(battery)[1] - (1 - Macro.storage_loss_fraction(battery)) * Macro.storage_level(battery)[T])
    else
        diff_storage_level[t] = Macro.value(Macro.storage_level(battery)[t] - (1 - Macro.storage_loss_fraction(battery)) * Macro.storage_level(battery)[t-1])
    end
end

power_in = battery_eff_up*Macro.value.(Macro.flow(battery.TEdges[:charge]).data);
power_out = (1/battery_eff_down)*Macro.value.(Macro.flow(battery.TEdges[:discharge]).data);

In [232]:
maximum(abs.(diff_storage_level - (power_in - power_out)))

1.268745108973235e-10